description of project, objective
disclaimer: you'll notice some inconsistency in how I handle certain operations/transformations/etc (e.g. convert to category vs convert to boolean vs convert to binary vs get_dummies). this is intended, as I'd like to show a variety of approaches to certain data situations/concerns/etc.

outline approach, plan of attack

base imports, descriptions, justification

In [1]:
import numpy as np
import pandas as pd
import math

where to obtain data, read in with pandas, inspect head and tail, important to check both train and test in case there are differences between the two that warrant consideration

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.tail()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
1458639,id2376096,2,2016-04-08 13:31:04,2016-04-08 13:44:02,4,-73.982201,40.745522,-73.994911,40.740170,N,778
1458640,id1049543,1,2016-01-10 07:35:15,2016-01-10 07:46:10,1,-74.000946,40.747379,-73.970184,40.796547,N,655
1458641,id2304944,2,2016-04-22 06:57:41,2016-04-22 07:10:25,1,-73.959129,40.768799,-74.004433,40.707371,N,764
1458642,id2714485,1,2016-01-05 15:56:26,2016-01-05 16:02:39,1,-73.982079,40.749062,-73.974632,40.757107,N,373
1458643,id1209952,1,2016-04-05 14:44:25,2016-04-05 14:47:43,1,-73.979538,40.781750,-73.972809,40.790585,N,198


In [4]:
test.tail()

,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag
625129,id3008929,1,2016-01-01 00:02:52,1,-74.003464,40.725105,-74.001251,40.733643,N
625130,id3700764,1,2016-01-01 00:01:52,1,-74.006363,40.743782,-73.953407,40.782467,N
625131,id2568735,1,2016-01-01 00:01:24,2,-73.972267,40.759865,-73.876602,40.748665,N
625132,id1384355,1,2016-01-01 00:00:28,1,-73.976501,40.733562,-73.854263,40.891788,N
625133,id0621643,2,2016-01-01 00:00:22,2,-73.981850,40.716881,-73.969330,40.769379,N


what data do we have? copy in descriptions from download page on kaggle

OR

.info to get column names, null counts, dtypes ... .unique to learn more about what's in each column

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
id                    1458644 non-null object
vendor_id             1458644 non-null int64
pickup_datetime       1458644 non-null object
dropoff_datetime      1458644 non-null object
passenger_count       1458644 non-null int64
pickup_longitude      1458644 non-null float64
pickup_latitude       1458644 non-null float64
dropoff_longitude     1458644 non-null float64
dropoff_latitude      1458644 non-null float64
store_and_fwd_flag    1458644 non-null object
trip_duration         1458644 non-null int64
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


no apparent null values. let's check for duplicates

In [6]:
print(any(train.duplicated()))
print(any(test.duplicated()))

False
False


id is described as unique identifier for each trip, so it's not going to add any value as a feature in our model (but we need to keep it, since it's the index for the submission). just for kicks and to demonstrate another approach to finding duplicates, check for uniqueness (explain methodology). I call .head so as to see what the ids look like -- otherwise, .max would have provided the answer I was looking for.

In [7]:
train.id.value_counts().sort_values(ascending=False).head()

id1857385    1
id3398959    1
id0792596    1
id2907918    1
id0979256    1
Name: id, dtype: int64

vendor id is a numeric, should be categorical (https://pandas-docs.github.io/pandas-docs-travis/categorical.html). let's check values, map to letters, for the sake of clarity.

In [8]:
train.vendor_id.unique()

array([2, 1])

In [9]:
test.vendor_id.unique()

array([1, 2])

In [10]:
vendor_map = {1:'A', 2:'B'}
train['vendor_id'] = train.vendor_id.map(vendor_map).astype('category')
test['vendor_id'] = test.vendor_id.map(vendor_map).astype('category')

In [11]:
train.dtypes

id                      object
vendor_id             category
pickup_datetime         object
dropoff_datetime        object
passenger_count          int64
pickup_longitude       float64
pickup_latitude        float64
dropoff_longitude      float64
dropoff_latitude       float64
store_and_fwd_flag      object
trip_duration            int64
dtype: object

In [12]:
test.dtypes

id                      object
vendor_id             category
pickup_datetime         object
passenger_count          int64
pickup_longitude       float64
pickup_latitude        float64
dropoff_longitude      float64
dropoff_latitude       float64
store_and_fwd_flag      object
dtype: object

based on description of store and fwd flag, it's a Y/N field. we could convert to a category or to a boolean, but a binary is more (directly) useful (? or is it just another option) in the models we'll be using. verify that there are no unexpected values, then convert to binary (explain method)

In [13]:
train.store_and_fwd_flag.unique()

array(['N', 'Y'], dtype=object)

In [14]:
test.store_and_fwd_flag.unique()

array(['N', 'Y'], dtype=object)

In [15]:
train['store_and_fwd_flag'] = (train.store_and_fwd_flag == 'Y').astype('int')
test['store_and_fwd_flag'] = (test.store_and_fwd_flag == 'Y').astype('int')

In [16]:
train.dtypes

id                      object
vendor_id             category
pickup_datetime         object
dropoff_datetime        object
passenger_count          int64
pickup_longitude       float64
pickup_latitude        float64
dropoff_longitude      float64
dropoff_latitude       float64
store_and_fwd_flag       int64
trip_duration            int64
dtype: object

In [17]:
test.dtypes

id                      object
vendor_id             category
pickup_datetime         object
passenger_count          int64
pickup_longitude       float64
pickup_latitude        float64
dropoff_longitude      float64
dropoff_latitude       float64
store_and_fwd_flag       int64
dtype: object

In [18]:
train.store_and_fwd_flag.unique()

array([0, 1])

In [19]:
test.store_and_fwd_flag.unique()

array([0, 1])

datetimes are not useful as objects, so we'll need to convert them to datetimes. refer to head/tail above to understand format (looks like one that pandas should be able to infer), then use pandas to convert

In [20]:
train['pickup_datetime'] = pd.to_datetime(train.pickup_datetime)
train['dropoff_datetime'] = pd.to_datetime(train.dropoff_datetime)
test['pickup_datetime'] = pd.to_datetime(test.pickup_datetime)

In [21]:
train.dtypes

id                            object
vendor_id                   category
pickup_datetime       datetime64[ns]
dropoff_datetime      datetime64[ns]
passenger_count                int64
pickup_longitude             float64
pickup_latitude              float64
dropoff_longitude            float64
dropoff_latitude             float64
store_and_fwd_flag             int64
trip_duration                  int64
dtype: object

In [22]:
test.dtypes

id                            object
vendor_id                   category
pickup_datetime       datetime64[ns]
passenger_count                int64
pickup_longitude             float64
pickup_latitude              float64
dropoff_longitude            float64
dropoff_latitude             float64
store_and_fwd_flag             int64
dtype: object

expecting dropoff_dt and trip duration to be redundant information (check kaggle descriptions?), but curious enough to verify before dismissing one of them

In [23]:
(train.dropoff_datetime - train.pickup_datetime).head()

0   00:07:35
1   00:11:03
2   00:35:24
3   00:07:09
4   00:07:15
dtype: timedelta64[ns]

In [24]:
pd.to_timedelta(train.trip_duration, unit='s').head()

0   00:07:35
1   00:11:03
2   00:35:24
3   00:07:09
4   00:07:15
Name: trip_duration, dtype: timedelta64[ns]

In [25]:
(train.dropoff_datetime - train.pickup_datetime == pd.to_timedelta(train.trip_duration, unit='s')).head()

0    True
1    True
2    True
3    True
4    True
dtype: bool

In [26]:
all(train.dropoff_datetime - train.pickup_datetime == pd.to_timedelta(train.trip_duration, unit='s'))

True

so, I'm going to drop the dropoff_dt from the train set

In [27]:
train.drop('dropoff_datetime', axis=1, inplace=True)

for pickup_dt, I'd like to look at this as a timeseries later, but for now the useful info needs to be extracted (thinking month, day of week, and time of day will be important, but also interested to see if day of month or minute of hour show any interesting trends). this will duplicate information, so I'll probably drop pickup_dt after we do some fun timeseries eda and before we select features for our models.

let's take a look first to see what timestamps are in our data

In [28]:
train.pickup_datetime.describe()

count                 1458644
unique                1380222
top       2016-03-04 08:07:34
freq                        5
first     2016-01-01 00:00:17
last      2016-06-30 23:59:39
Name: pickup_datetime, dtype: object

In [29]:
test.pickup_datetime.describe()

count                  625134
unique                 610581
top       2016-02-04 19:54:06
freq                        4
first     2016-01-01 00:00:22
last      2016-06-30 23:59:58
Name: pickup_datetime, dtype: object

interesting that we only have data for 6 months. note that all data is from 2016. also note frequency -- some timestamps are duplicated. now extract ...

In [30]:
train['pickup_month'] = train.pickup_datetime.dt.strftime('%b').astype('category')
train['pickup_dayofmonth'] = train.pickup_datetime.dt.strftime('%d').astype('category')
train['pickup_dayofweek'] = train.pickup_datetime.dt.strftime('%a').astype('category')
train['pickup_hourofday'] = train.pickup_datetime.dt.strftime('%H').astype('category')
train['pickup_minuteofhour'] = train.pickup_datetime.dt.strftime('%M').astype('category')

test['pickup_month'] = test.pickup_datetime.dt.strftime('%b').astype('category')
test['pickup_dayofmonth'] = test.pickup_datetime.dt.strftime('%d').astype('category')
test['pickup_dayofweek'] = test.pickup_datetime.dt.strftime('%a').astype('category')
test['pickup_hourofday'] = test.pickup_datetime.dt.strftime('%H').astype('category')
test['pickup_minuteofhour'] = test.pickup_datetime.dt.strftime('%M').astype('category')

In [31]:
print(train.dtypes)
train.head()

id                             object
vendor_id                    category
pickup_datetime        datetime64[ns]
passenger_count                 int64
pickup_longitude              float64
pickup_latitude               float64
dropoff_longitude             float64
dropoff_latitude              float64
store_and_fwd_flag              int64
trip_duration                   int64
pickup_month                 category
pickup_dayofmonth            category
pickup_dayofweek             category
pickup_hourofday             category
pickup_minuteofhour          category
dtype: object


,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_month,pickup_dayofmonth,pickup_dayofweek,pickup_hourofday,pickup_minuteofhour
0,id2875421,B,2016-03-14 17:24:55,1,-73.982155,40.767937,-73.964630,40.765602,0,455,Mar,14,Mon,17,24
1,id2377394,A,2016-06-12 00:43:35,1,-73.980415,40.738564,-73.999481,40.731152,0,663,Jun,12,Sun,00,43
2,id3858529,B,2016-01-19 11:35:24,1,-73.979027,40.763939,-74.005333,40.710087,0,2124,Jan,19,Tue,11,35
3,id3504673,B,2016-04-06 19:32:31,1,-74.010040,40.719971,-74.012268,40.706718,0,429,Apr,06,Wed,19,32
4,id2181028,B,2016-03-26 13:30:55,1,-73.973053,40.793209,-73.972923,40.782520,0,435,Mar,26,Sat,13,30


In [32]:
print(test.dtypes)
test.head()

id                             object
vendor_id                    category
pickup_datetime        datetime64[ns]
passenger_count                 int64
pickup_longitude              float64
pickup_latitude               float64
dropoff_longitude             float64
dropoff_latitude              float64
store_and_fwd_flag              int64
pickup_month                 category
pickup_dayofmonth            category
pickup_dayofweek             category
pickup_hourofday             category
pickup_minuteofhour          category
dtype: object


,id,vendor_id,pickup_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,pickup_month,pickup_dayofmonth,pickup_dayofweek,pickup_hourofday,pickup_minuteofhour
0,id3004672,A,2016-06-30 23:59:58,1,-73.988129,40.732029,-73.990173,40.756680,0,Jun,30,Thu,23,59
1,id3505355,A,2016-06-30 23:59:53,1,-73.964203,40.679993,-73.959808,40.655403,0,Jun,30,Thu,23,59
2,id1217141,A,2016-06-30 23:59:47,1,-73.997437,40.737583,-73.986160,40.729523,0,Jun,30,Thu,23,59
3,id2150126,B,2016-06-30 23:59:41,1,-73.956070,40.771900,-73.986427,40.730469,0,Jun,30,Thu,23,59
4,id1598245,A,2016-06-30 23:59:33,1,-73.970215,40.761475,-73.961510,40.755890,0,Jun,30,Thu,23,59


this is a good place to take a break. data gathering was way simple, bc kaggle. summarize cleaning effort. next, we're going to do some EDA to gain some understanding of how our label and features are distributed, check for outliers, and identify any additional features we might be able to work into our data.

but actually, because I already know I'm going to need to work out trip distance, let's just create that feature now. there are several different approaches I could take: difference of coordinates in each direction for rough manhattan estimate (or I could calculate the euclidean coordinate distance, which would be an even rougher estimate), something more meaniningful like miles or km (based on haversine calculation) again for a rough manhattan estimate, or I could hit up an API like Google Maps (or OSRM or ???) for something way more accurate. the roughness of the manhattan distance won't be so bad on the island, but off-island, roads may be less grid-ish/y/like. anyway, I'm going with haversine manhattan kms.

In [33]:
# earth_radius = 6371
# ns_train = []
# for i in abs(train.dropoff_latitude - train.pickup_latitude) :
#     ns_train.append(earth_radius * math.radians(i))
# ew_train = []
# for k in abs(train.dropoff_longitude - train.pickup_longitude) :
#     for j in ((train.pickup_latitude + train.dropoff_latitude) / 2) :
#         ew_train.append(earth_radius * math.cos(math.radians(j)) * k)
# train['ns_dist'] = ns_train
# train['ew_dist'] = ew_train

# ns_test = []
# for i in abs(test.dropoff_latitude - test.pickup_latitude) :
#     ns_test.append(earth_radius * math.radians(i))
# ew_test = []
# for k in abs(test.dropoff_longitude - test.pickup_longitude) :
#     for j in ((test.pickup_latitude + test.dropoff_latitude) / 2) :
#         ew_test.append(earth_radius * math.cos(math.radians(j)) * k)
# test['ns_dist'] = ns_test
# test['ew_dist'] = ew_test

not enough memory for that, apparently. let's do this instead, for now

In [37]:
train['lon_dist'] = train.dropoff_longitude - train.pickup_longitude
train['lat_dist'] = train.dropoff_latitude - train.pickup_latitude
test['lon_dist'] = test.dropoff_longitude - test.pickup_longitude
test['lat_dist'] = test.dropoff_latitude - test.pickup_latitude

In [41]:
train['lon_dist'] = [math.radians(i) for i in train.lon_dist]
train['lat_dist'] = [math.radians(i) for i in train.lat_dist]
test['lon_dist'] = [math.radians(i) for i in test.lon_dist]
test['lat_dist'] = [math.radians(i) for i in test.lat_dist]

In [42]:
earth_radius = 6371
nyc_lat = math.radians(40.7128)
train['lon_dist'] = [earth_radius * math.cos(nyc_lat) * i for i in train.lon_dist]
train['lat_dist'] = [earth_radius * i for i in train.lat_dist]
test['lon_dist'] = [earth_radius * math.cos(nyc_lat) * i for i in test.lon_dist]
test['lat_dist'] = [earth_radius * i for i in test.lat_dist]

In [49]:
train['man_dist'] = abs(train.lon_dist) + abs(train.lat_dist)
test['man_dist'] = abs(test.lon_dist) + abs(test.lat_dist)